In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import mne
import pickle
import seaborn as sns
import pandas as pd
from scipy import stats
from statannotations.Annotator import Annotator

plt.rcParams.update({'font.size': 14})

# PFI

In [64]:
# Load meg channel config
dataset_path = os.path.join('cichy_data', 'subj0', 'MEG2_subj01_sess01_tsss_mc-3.fif')
raw = mne.io.read_raw_fif(dataset_path, preload=True)
chn_type = 'mag'
raw = raw.pick(chn_type)

# load pfi data
path = os.path.join('..', 'results', 'cichy_epoched', 'all_noshuffle_wavenetclass_semb10_drop0.4', 'groupPFI',
                    'val_loss_PFIch4.npy')
pfi = np.load(open(path, 'rb'))
pfi = pfi[0, 0, 0] - pfi[:, :, 1:]

Opening raw data file cichy_data/subj0/MEG2_subj01_sess01_tsss_mc-3.fif...
    Range : 4930000 ... 5162999 =   4930.000 ...  5162.999 secs
Ready.
Reading 0 ... 232999  =      0.000 ...   232.999 secs...


/var/folders/nv/wjmf18wd5_j38vg9v0cthl5h0000gn/T/ipykernel_2304/202119110.py:3: RuntimeWarning: This filename (cichy_data/subj0/MEG2_subj01_sess01_tsss_mc-3.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(dataset_path, preload=True)


In [65]:
# PFIts
path = os.path.join('..', 'results', 'cichy_epoched', 'all_noshuffle_wavenetclass_semb10_drop0.4', 'groupPFI',
                    'val_loss_PFIts.npy')
pfits = np.load(open(path, 'rb'))
pfits = pfits[0, 0] - pfits

In [66]:
# times array
times = np.array([np.arange(-48, 872, 4) for _ in range(pfi.shape[0])])
times = np.array([times.reshape(-1) for _ in range(pfi.shape[2])]).T
times = times.reshape(-1)

# channels array
pfi_pd = pfi.reshape(-1, pfi.shape[2])

# magnitudes for color hues
mags = np.abs(np.mean(pfi, axis=(0, 1)))
mags = np.array([mags/np.max(mags[:-1]) for _ in range(pfi_pd.shape[0])])
mags = mags.reshape(-1)

pfi_pd = pfi_pd.reshape(-1)

In [67]:
# put everything in a pd dataframe
pd_dict = {'Accuracy loss': pfi_pd, 'Time (ms)': times, 'relative magnitude': mags, 'channels': ['individual']*mags.shape[0]}
pfi_pd = pd.DataFrame(pd_dict)

In [68]:
times = np.array([np.arange(-48, 872, 4) for _ in range(pfits.shape[0])])
pfits_pd = pfits.reshape(-1)
times = times.reshape(-1)
mags_ts = [np.max(mags)] * times.shape[0]
channels = ['aggregate'] * times.shape[0]
inds = list(range(mags.shape[0], mags.shape[0]+times.shape[0]))
pfits_pd = pd.DataFrame({'Accuracy loss': pfits_pd, 'Time (ms)': times, 'relative magnitude': mags_ts, 'channels': channels}, index=inds)
pfi_pd = pd.concat((pfi_pd, pfits_pd))

In [70]:
%matplotlib widget
pfi_plot = sns.relplot(
    data=pfits_pd, kind="line",
    x="Time (ms)", y="Accuracy loss", hue='relative magnitude', n_boot=10, legend=None, aspect=1.1
)
plt.savefig('/Users/ricsi/GoogleDrive/Projektek/The Brain Project/oxford/writings/neurips_figures/temporal_PFI.pdf', format='pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
pfi_pd

,Accuracy loss,Time (ms),relative magnitude,channels
0,0.000476,-48,0.048967,individual
1,0.000476,-48,0.055159,individual
2,0.001428,-48,0.102658,individual
3,-0.000381,-48,0.040458,individual
4,-0.001143,-48,0.061394,individual
...,...,...,...,...
236895,0.017140,852,1.000000,aggregate
236896,0.017711,856,1.000000,aggregate
236897,0.016759,860,1.000000,aggregate
236898,0.016187,864,1.000000,aggregate


In [8]:
%matplotlib widget
pfi_plot = sns.relplot(
    data=pfi_pd, kind="line", size='channels', style='channels', size_order=['aggregate', 'individual'],
    x="Time (ms)", y="Accuracy loss", hue='relative magnitude', n_boot=10, legend='brief', palette='Reds', aspect=2
)

handles, labels = plt.gca().get_legend_handles_labels()
pfi_plot._legend.remove()
plt.legend(handles[-3:], labels[-3:])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
plt.savefig('/Users/ricsi/GoogleDrive/Projektek/The Brain Project/oxford/writings/neurips_figures/channelwise_PFI.svg', format='svg')

# Accuracies

In [2]:
df = pd.read_csv('excel_data.txt', sep="\t", header=0)
df = df.stack().reset_index()
df = df.rename(columns={'level_0': 'subject', 'level_1': 'model', 0: 'Validation accuracy'})

model_type = []
for i in range(len(df)):
    if 'emb' in df['model'][i]:
        model_type.append('group\nembedding')
    elif 'group' in df['model'][i]:
        model_type.append('group')
    else:
        model_type.append('subject')
df['model type'] = model_type

In [3]:
df[df['model']=='nonlin-group-emb']

,subject,model,Validation accuracy,model type
5,0,nonlin-group-emb,0.490662,group\nembedding
12,1,nonlin-group-emb,0.318117,group\nembedding
19,2,nonlin-group-emb,0.112696,group\nembedding
26,3,nonlin-group-emb,0.522857,group\nembedding
33,4,nonlin-group-emb,0.705128,group\nembedding
40,5,nonlin-group-emb,0.471510,group\nembedding
47,6,nonlin-group-emb,0.637269,group\nembedding
54,7,nonlin-group-emb,0.212554,group\nembedding
61,8,nonlin-group-emb,0.464950,group\nembedding
68,9,nonlin-group-emb,0.454416,group\nembedding


In [5]:
%matplotlib widget
'''
rows = df.loc[df['model type'] == 'subject']
g = sns.catplot(kind="violin", data=rows, cut=0, ci=None, scale='width', y='Validation accuracy', x='model')

ax = g.axes[0][0]
ax.axhline(0.008, ls='-', color='black', label='chance')
ax.axhline(0.33, ls='--', color='black', label='peak sliding accuracy')
plt.ylim(0, 1)
plt.legend()
plt.xlabel('')
plt.savefig('/Users/ricsi/GoogleDrive/Projektek/The Brain Project/oxford/writings/neurips_figures/subject_acc.svg', format='svg')
'''

plt.rcParams.update({'font.size': 14})
# Putting the parameters in a dictionary avoids code duplication
# since we use the same for `sns.boxplot` and `Annotator` calls
plot_params = {
    'kind':    'violin',
    'aspect':  2.1,
    'cut':     0,
    'ci':      None,
    'scale':   'area',
    'hue':     'model type',
    'dodge':   False,
    'data':    df,
    'x':       'model',
    'y':       'Validation accuracy',
    'legend':  None
}

g = sns.catplot(**plot_params)

ax = g.axes[0][0]
ax.axhline(0.008, ls='-', color='black', label='chance')
plt.ylim(0, 1)
plt.xlabel('')
plt.text(6.6,0.015,'chance')
plt.xticks(plt.xticks()[0], ['linear\nsubject',
                             'nonlinear\nsubject',
                             'linear\ngroup',
                             'nonlinear\ngroup',
                             'linear\ngroup-emb',
                             'nonlinear\ngroup-emb',
                             'nonlinear\ngroup-emb\nfinetuned'])

ymin = 0.02
ymax = 0.7
alpha = 0.5
dash = '--'
color = 'red'
ax.axvline(1.5, ymin, ymax, ls=dash, color=color, alpha=alpha)
ax.axvline(3.5, ymin, ymax, ls=dash, color=color, alpha=alpha)
plt.text(0.22,0.87,'subject models')
plt.text(2,0.8,'group models')
plt.text(3.7,0.8,'group models\nwith embedding')

# which pairs to computer stats on
pairs = [('lin-subject', 'nonlin-subject'),
         ('lin-subject', 'nonlin-group-emb'),
         ('lin-subject', 'nonlin-group-emb finetuned'),
         ('nonlin-group-emb', 'nonlin-group-emb finetuned'),
         ('nonlin-group-emb', 'nonlin-group')]

# Add statistics annotations
annotator = Annotator(ax, pairs, data=df, x='model', y='Validation accuracy')
annotator.configure(test='Wilcoxon', verbose=True, line_offset_to_group=10).apply_and_annotate()

plt.savefig('/Users/ricsi/GoogleDrive/Projektek/The Brain Project/oxford/writings/neurips_figures/group_acc.pdf', format='pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

p-value annotation legend:
      ns: p <= 1.00e+00
       *: 1.00e-02 < p <= 5.00e-02
      **: 1.00e-03 < p <= 1.00e-02
     ***: 1.00e-04 < p <= 1.00e-03
    ****: p <= 1.00e-04

lin-subject vs. nonlin-subject: Wilcoxon test (paired samples), P_val:6.104e-04 Stat=5.000e+00
nonlin-group-emb vs. nonlin-group-emb finetuned: Wilcoxon test (paired samples), P_val:6.104e-05 Stat=0.000e+00
nonlin-group vs. nonlin-group-emb: Wilcoxon test (paired samples), P_val:6.104e-05 Stat=0.000e+00
lin-subject vs. nonlin-group-emb: Wilcoxon test (paired samples), P_val:2.155e-02 Stat=2.000e+01
lin-subject vs. nonlin-group-emb finetuned: Wilcoxon test (paired samples), P_val:8.545e-04 Stat=6.000e+00


# Generalization to new subject

In [76]:
path = os.path.join('..', 'results', 'cichy_epoched', 'indiv_wavenetlinear_MNN', 'val_loss_general.npy')
accs = np.load(path)
train1 = [0.591525424, 0.303672316, 0.121468925, 0.680790966, 0.885593221, 0.662429377, 0.730225995, 0.159604517, 0.579096052, 0.627118642, 0.223163842, 0.151129942, 0.06497175, 0.483050848, 0.412429377]
accs = np.concatenate((accs, np.array(train1).reshape(-1, 1)), axis=1)
chance = [0.00847] * 15
accs = np.concatenate((np.array(chance).reshape(-1, 1), accs), axis=1)

In [77]:
accs_df = pd.DataFrame(accs)
accs_df = accs_df.stack().reset_index()
accs_df = accs_df.rename(columns={'level_0': 'subject', 'level_1': 'Training ratio', 0: 'Validation accuracy'})
accs_df['level'] = ['subject'] * len(accs_df)

In [159]:
accs_df

,level_0,level_1,0
0,0,0,0.008470
1,0,1,0.008475
2,0,2,0.013559
3,0,3,0.050847
4,0,4,0.116949
...,...,...,...
160,14,6,0.162429
161,14,7,0.213277
162,14,8,0.283898
163,14,9,0.331921


In [78]:
def create_df(accsg, level):
    # need to get actual subjects
    order = [10, 7, 3, 11, 8, 4, 12, 9, 5, 13, 1, 14, 2, 6, 0]
    accsg_df = pd.DataFrame(accsg[order, :])
    accsg_df = accsg_df.stack().reset_index()
    accsg_df = accsg_df.rename(columns={'level_0': 'subject', 'level_1': 'Training ratio', 0: 'Validation accuracy'})
    accsg_df['level'] = [level] * len(accsg_df)
    
    return accsg_df

In [79]:
path = os.path.join('..', 'results', 'cichy_epoched', 'all_wavenet_semb_general', 'val_loss_general.npy')
accsg = np.load(path)
group_emb = create_df(accsg, 'group-emb')

path = os.path.join('..', 'results', 'cichy_epoched', 'all_wavenet_general', 'val_loss_general.npy')
accsg = np.load(path)
group = create_df(accsg, 'group')

path = os.path.join('..', 'results', 'cichy_epoched', 'all_wavenet_semb_general', 'val_loss_subs_general.npy')
accsg = np.mean(np.load(path), axis=2)
group_emb_all = create_df(accsg, 'group-emb all')

path = os.path.join('..', 'results', 'cichy_epoched', 'all_wavenet_general', 'val_loss_subs_general.npy')
accsg = np.mean(np.load(path), axis=2)
group_all = create_df(accsg, 'group all')

In [80]:
df = pd.concat((accs_df, group_emb, group), ignore_index=True)

In [12]:
df[(df['subject'] == 14) & (df['level'] == 'group')]

,subject,Training ratio,Validation accuracy,level
319,14,0,0.043785,group
320,14,1,0.103107,group
321,14,2,0.152542,group
322,14,3,0.241525,group
323,14,4,0.210452,group
324,14,5,0.290960,group
325,14,6,0.314972,group
326,14,7,0.350282,group
327,14,8,0.411017,group
328,14,9,0.436441,group


In [19]:
df

,subject,Training ratio,Validation accuracy,level
0,0,0.0,0.008470,subject
1,0,0.1,0.008475,subject
2,0,0.2,0.013559,subject
3,0,0.3,0.050847,subject
4,0,0.4,0.116949,subject
...,...,...,...,...
325,14,0.6,0.314972,group
326,14,0.7,0.350282,group
327,14,0.8,0.411017,group
328,14,0.9,0.436441,group


In [11]:
df[df['subject'] == 14]

,subject,Training ratio,Validation accuracy,level
154,14,0,0.008470,subject
155,14,1,0.009887,subject
156,14,2,0.012712,subject
157,14,3,0.028249,subject
158,14,4,0.069209,subject
159,14,5,0.128531,subject
160,14,6,0.162429,subject
161,14,7,0.213277,subject
162,14,8,0.283898,subject
163,14,9,0.331921,subject


In [81]:
df['Training ratio'] = df['Training ratio'].astype(float)/10

In [29]:
p_values = []
for i in range(11):
    test1 = df['Validation accuracy'][(df['Training ratio'] == i) & (df['level'] == 'group all')]
    test2 = df['Validation accuracy'][(df['Training ratio'] == i) & (df['level'] == 'group-emb all')]
    
    p_values.append(stats.ttest_rel(test1, test2)[1] * 11)

In [30]:
p_values

[0.0060336492775827375,
 1.351510411629857,
 0.03169837884573075,
 4.3785466680206016,
 2.918205427079515,
 0.10703676926544689,
 0.020678270815296832,
 0.055122900986178884,
 0.11919219652472021,
 0.23726568570982257,
 nan]

In [83]:
%matplotlib widget
g = sns.relplot(
    data=df, kind="line", hue='level',
    x="Training ratio", y="Validation accuracy", n_boot=1000, aspect=1.2, ci=95
)
ax = g.axes[0][0]
plt.axhline(0.6, 0.02, 0.7, color='black')
plt.text(0,0.61,'group>subject (p<0.05)')
plt.ylim(0, 0.6)
plt.xlabel('Training set ratio')



ax.axhline(0.008, ls='-', color='black', label='chance')
ax.legend(loc='upper left')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [84]:
plt.savefig('/Users/ricsi/GoogleDrive/Projektek/The Brain Project/oxford/writings/neurips_figures/generalization.pdf', format='pdf')